In [1]:
import Utils
Utils.set_seed(Utils.seed)

import Dataloader,  Model
import torch
import pywt
import pandas               as  pd
import numpy                as  np
import matplotlib.pyplot    as  plt
import torch.nn             as  nn


# Hyper Parameters

In [2]:
# model
# CNN
CNN_inChannel = 1
CNN_outChannel = 10

# Continuous Wavelet transform (CWT)

In [3]:
scales = np.arange(1, Utils.Scales)

for wavelet in reversed(Utils.wavelet_lis):
    # Perform Continuous Wavelet Transform
    print(wavelet)
    coefficients, frequencies = pywt.cwt(Dataloader.X_numpy[1,:Utils.input_horizon], scales, wavelet)

    # # Plot the results
    # plt.figure(figsize=(10, 6))
    # plt.imshow(np.abs(coefficients), extent=[0, 1, 1, Scales], aspect='auto', cmap='jet')
    # # plt.colorbar(label='Magnitude')
    # plt.title('Continuous Wavelet Transform')
    # plt.xlabel('Time')
    # plt.ylabel('Scale')
    # plt.show()

if Utils.Coefficient_Real:
    coefficients = np.abs(coefficients)

cmor1-1.0
gaus1
morl
mexh


# Model

in ``` h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) ``` x is input and data will be generated in device

In [4]:
class Structure(nn.Module):
    def __init__(self,):
        super(Structure, self).__init__()
        self.LSTM = Model.LSTMModel(Utils.input_horizon                 ).to(device = Utils.Device)
        self.CNN  = Model.InceptionBlock(CNN_inChannel,CNN_outChannel   ).to(device = Utils.Device)

        # CWT
        self.scales = np.arange(1, Utils.Scales)

        # MLP
        self.Classifier = nn.Linear(200,1)

    def _CWT(self,  x:np.array):
        coefficients, _ = pywt.cwt(x, self.scales, Utils.wavelet)
        if Utils.Coefficient_Real:
            coefficients = np.abs(coefficients)
        return coefficients

    def forward(self, x:np.array):
        coefficients = self._CWT(x)

        _L = torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device = Utils.Device)
        _L_out = self.LSTM(_L)

        _C = torch.tensor(coefficients, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device = Utils.Device)
        _C_out = inception_block = self.CNN(_C)

        out = torch.cat([_C_out,_L_out], dim=1)
        out = self.Classifier(out)
        return out

In [7]:
test = Structure().to(device = Utils.Device)

In [8]:
gg = test(Dataloader.X_numpy[1,:Utils.input_horizon])

In [10]:
gg

tensor([[-0.1311]], device='cuda:0', grad_fn=<AddmmBackward0>)